# Filter out only team USA and predict the current points

In [1]:
#general
import io
import os
import shutil
import requests
import base64

#pandas
import pandas as pd

#TensorFlow
import tensorflow as tf
#numpy
import numpy as np

#matplotlib.pylot
import matplotlib.pyplot as plt

#sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

#keras
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.layers.core import Dense, Activation

Using TensorFlow backend.


In [2]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os
import requests
import base64


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low

In [3]:
#path
path = "/Users/petronillagriffith/desktop/Fifa/fifa"

#import file
filename = os.path.join(path, "USA.csv")

#names each column
names = ['rank', 'country', 'total_points', 'previous_points', 'rank_change', 'cur_year_avg', 
          'cur_year_avg_weighted', 'last_year_avg', 'last_year_avg_weighted', 'two_year_ago_avg', 
          'two_year_ago_avg_weighted', 'three_year_ago_avg_', 'three_year_ago_avg_weighted', 'confederation', 'rank_date']

#making a DataFrame
df = pd.read_csv(filename, encoding = "ISO-8859-1", sep=',', na_values=['-1'], index_col=False)

#writes out "Starting File:"
print("Starting File:")


print(df[20:100])


print()

#writes out "Ending File:"
print("Ending File:")

print(df[-10:1])

Starting File:
    rank country_full country_abrv  total_points  previous_points  \
20    28          USA          USA        779.25              733   
21    29          USA          USA        779.25              779   
22    28          USA          USA        798.28              779   
23    22          USA          USA        865.10              798   
24    19          USA          USA        959.80              865   
25    13          USA          USA        995.75              960   
26    13          USA          USA       1039.80              996   
27    14          USA          USA       1018.71             1040   
28    14          USA          USA       1018.71             1019   
29    14          USA          USA       1018.71             1019   
30    13          USA          USA       1043.70             1019   
31    14          USA          USA       1017.32             1044   
32    13          USA          USA       1014.50             1017   
33    14          U

In [4]:
df.head(5)

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_avg_weighted,three_year_ago_avg,three_year_ago_avg_weighted,confederation,rank_date
0,28,USA,USA,791.44,777,2.0,335.41,335.41,430.01,215.01,597.61,179.28,308.68,61.74,CONCACAF,8/24/11
1,31,USA,USA,741.45,791,-3.0,290.69,290.69,380.25,190.13,604.26,181.28,396.74,79.35,CONCACAF,9/21/11
2,34,USA,USA,719.61,741,-3.0,298.14,298.14,308.72,154.36,634.66,190.40,383.53,76.71,CONCACAF,10/19/11
3,34,USA,USA,720.85,720,0.0,285.53,285.53,358.73,179.37,583.78,175.13,404.11,80.82,CONCACAF,11/23/11
4,34,USA,USA,720.85,721,0.0,285.53,285.53,358.73,179.37,583.78,175.13,404.11,80.82,CONCACAF,12/21/11


In [5]:
a = df['country_full'].unique().tolist()

byte_offset_map = {}

for i, id in enumerate(a): byte_offset_map[id] = i
    
df['country_full_class'] = df['country_full'].map(lambda x: byte_offset_map[x])

In [6]:
a = df['country_abrv'].unique().tolist()

byte_offset_map = {}

for i, id in enumerate(a): byte_offset_map[id] = i
    
df['country_abrv_class'] = df['country_abrv'].map(lambda x: byte_offset_map[x])

In [7]:
a = df['confederation'].unique().tolist()

byte_offset_map = {}

for i, id in enumerate(a): byte_offset_map[id] = i
    
df['confederation_class'] = df['confederation'].map(lambda x: byte_offset_map[x])

In [8]:
a = df['rank_date'].unique().tolist()

byte_offset_map = {}

for i, id in enumerate(a): byte_offset_map[id] = i
    
df['rank_date_class'] = df['rank_date'].map(lambda x: byte_offset_map[x])

In [9]:
df

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_avg_weighted,three_year_ago_avg,three_year_ago_avg_weighted,confederation,rank_date,country_full_class,country_abrv_class,confederation_class,rank_date_class
0,28,USA,USA,791.44,777,2.0,335.41,335.41,430.01,215.01,597.61,179.28,308.68,61.74,CONCACAF,8/24/11,0,0,0,0
1,31,USA,USA,741.45,791,-3.0,290.69,290.69,380.25,190.13,604.26,181.28,396.74,79.35,CONCACAF,9/21/11,0,0,0,1
2,34,USA,USA,719.61,741,-3.0,298.14,298.14,308.72,154.36,634.66,190.40,383.53,76.71,CONCACAF,10/19/11,0,0,0,2
3,34,USA,USA,720.85,720,0.0,285.53,285.53,358.73,179.37,583.78,175.13,404.11,80.82,CONCACAF,11/23/11,0,0,0,3
4,34,USA,USA,720.85,721,0.0,285.53,285.53,358.73,179.37,583.78,175.13,404.11,80.82,CONCACAF,12/21/11,0,0,0,4
5,33,USA,USA,720.85,721,1.0,285.53,285.53,358.73,179.37,583.78,175.13,404.11,80.82,CONCACAF,1/18/12,0,0,0,5
6,31,USA,USA,750.47,721,2.0,308.64,308.64,371.15,185.58,539.98,162.00,471.24,94.25,CONCACAF,2/15/12,0,0,0,6
7,27,USA,USA,771.60,750,4.0,321.57,321.57,405.60,202.80,509.94,152.98,471.24,94.25,CONCACAF,3/7/12,0,0,0,7
8,29,USA,USA,778.69,772,-2.0,350.25,350.25,360.82,180.41,510.95,153.29,473.71,94.74,CONCACAF,4/11/12,0,0,0,8
9,29,USA,USA,778.69,779,0.0,350.25,350.25,360.82,180.41,510.95,153.29,473.71,94.74,CONCACAF,5/9/12,0,0,0,9


In [10]:
#Drops the 'country' column from the data set
df = df.drop(['country_full'], axis=1)

In [11]:
#Drops the 'country' column from the data set
df = df.drop(['country_abrv'], axis=1)

In [12]:
#Drops the 'confederation' column from the data set
df = df.drop(['confederation'], axis=1)

In [13]:
#Drops the 'confederation' column from the data set
df = df.drop(['rank_date'], axis=1)

In [14]:
df.head(5)

,rank,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_avg_weighted,three_year_ago_avg,three_year_ago_avg_weighted,country_full_class,country_abrv_class,confederation_class,rank_date_class
0,28,791.44,777,2.0,335.41,335.41,430.01,215.01,597.61,179.28,308.68,61.74,0,0,0,0
1,31,741.45,791,-3.0,290.69,290.69,380.25,190.13,604.26,181.28,396.74,79.35,0,0,0,1
2,34,719.61,741,-3.0,298.14,298.14,308.72,154.36,634.66,190.40,383.53,76.71,0,0,0,2
3,34,720.85,720,0.0,285.53,285.53,358.73,179.37,583.78,175.13,404.11,80.82,0,0,0,3
4,34,720.85,721,0.0,285.53,285.53,358.73,179.37,583.78,175.13,404.11,80.82,0,0,0,4


In [15]:
len(df)

83

In [16]:
max = df.loc[df['country_full_class'].idxmax()]

In [17]:
total_country = max['country_full_class']

In [18]:
total_country

0.0

In [19]:
df.dtypes

rank                             int64
total_points                   float64
previous_points                  int64
rank_change                    float64
cur_year_avg                   float64
cur_year_avg_weighted          float64
last_year_avg                  float64
last_year_avg_weighted         float64
two_year_ago_avg               float64
two_year_ago_avg_weighted      float64
three_year_ago_avg             float64
three_year_ago_avg_weighted    float64
country_full_class               int64
country_abrv_class               int64
confederation_class              int64
rank_date_class                  int64
dtype: object

In [38]:
df.isnull().values.any()

False

In [21]:
#x, y = to_xy(df,'total_points')

#y = df['total_points']


#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
#print(x_train.shape)
#print(x_test.shape)


In [22]:
x = int(len(df)*.7)
pt_to_split = int(x)
print(pt_to_split)

58


In [23]:
x = df
x_train= df[:pt_to_split]
x_test = df[pt_to_split:]
print(x_train.shape)
print(x_test.shape)

(58, 16)
(25, 16)


In [24]:
y = df['total_points']
y_train = y[:pt_to_split]
y_test = y [pt_to_split:]
print(y_train.shape)
print(y_test.shape)

(58,)
(25,)


In [25]:
#df = df.drop(['rank_change'], axis=1)

In [26]:
nan_rows = df[df.isnull().T.any().T]
nan_rows

,rank,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_avg_weighted,three_year_ago_avg,three_year_ago_avg_weighted,country_full_class,country_abrv_class,confederation_class,rank_date_class
16,28,781.58,776,NaN,414.44,414.44,285.53,142.76,358.73,107.62,583.78,116.76,0,0,0,16
19,33,732.77,736,NaN,348.32,348.32,321.57,160.78,405.60,121.68,509.94,101.99,0,0,0,19
21,29,779.25,779,NaN,393.69,393.69,350.25,175.13,360.82,108.24,510.95,102.19,0,0,0,21
27,14,1018.71,1040,NaN,654.09,654.09,414.44,207.22,285.53,85.66,358.73,71.75,0,0,0,27
31,14,1017.32,1044,NaN,665.57,665.57,348.32,174.16,321.57,96.47,405.60,81.12,0,0,0,31
33,14,1014.50,1015,NaN,640.42,640.42,393.69,196.84,350.25,105.08,360.82,72.16,0,0,0,33
43,32,827.97,824,NaN,326.38,326.38,665.57,332.78,348.32,104.50,321.57,64.31,0,0,0,43
45,28,824.50,815,NaN,316.14,316.14,640.42,320.21,393.69,118.11,350.25,70.05,0,0,0,45
50,29,807.02,823,NaN,317.94,317.94,444.73,222.36,662.73,198.82,339.52,67.90,0,0,0,50
60,26,848.13,855,NaN,410.23,410.23,336.44,168.22,466.52,139.96,648.60,129.72,0,0,0,60


In [27]:
len(nan_rows)

15

In [28]:
df = df[pd.notnull(df['rank_change'])]

In [29]:
len(nan_rows)

15

In [30]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(58, 16)
(25, 16)
(58,)
(25,)


In [31]:
y_train = y_train.values.reshape(58,1)
y_test = y_test.values.reshape(25,1)

In [32]:
x = int(len(df)*.7)
pt_to_split = int(x)

In [33]:
df_train = df[df['rank_date_class']<pt_to_split]
df_test = df[df['rank_date_class']>=pt_to_split]

spots_train = df_train['total_points'].tolist()
spots_test = df_test['total_points'].tolist()

print("Training set has {} observations.".format(len(spots_train)))
print("Test set has {} observations.".format(len(spots_test)))

Training set has 39 observations.
Test set has 29 observations.


In [34]:
import numpy as np

def to_sequences(seq_size, obs):
    x = []
    y = []
    
    for i in range(len(obs)-SEQUENCE_SIZE-1):
        #print(i)
        window = obs[i: (i+SEQUENCE_SIZE)]
        after_window = obs[i+SEQUENCE_SIZE]
        window = [[x] for x in window]
        x.append(window)
        y.append(after_window)
    return np.array(x),np.array(y)

SEQUENCE_SIZE = 5

x_train,y_train = to_sequences(SEQUENCE_SIZE,spots_train)
x_test,y_test = to_sequences(SEQUENCE_SIZE,spots_test)

print("Shape of training set: {}".format(x_train.shape))
print("Shape of test set: {}".format(x_test.shape))

Shape of training set: (33, 5, 1)
Shape of test set: (23, 5, 1)


In [35]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.callbacks import EarlyStopping
import numpy as np

print('Build model...')
model = Sequential()
model.add(LSTM(64, dropout=0.0, recurrent_dropout=0.0, input_shape=(None, 1)))
model.add(Dense(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
print('Train...')
checkpointer = ModelCheckpoint(filepath="best_weights.hdf5", verbose=0, save_best_only=True) # save best model
model.summary()

model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=1,epochs=1000)
model.load_weights('best_weights.hdf5') # load weights from best model

Build model...
Train...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________
Train on 33 samples, validate on 23 samples
Epoch 1/1000
33/33 [==============================] - 0s 11ms/step - loss: 739200.8371 - val_loss: 696972.8125
Epoch 2/1000
33/33 [==============================] - 0s 1ms/step - loss: 738656.5739 - val_loss: 696536.1250
Epoch 3/1000
33/33 [==============================] - 0s 1ms/step - loss: 738210.2595 - val_loss: 696102.3125
Epo

33/33 [==============================] - 0s 1ms/step - loss: 574699.6307 - val_loss: 534541.1875
Epoch 141/1000
33/33 [==============================] - 0s 1ms/step - loss: 572579.6004 - val_loss: 532497.4375
Epoch 142/1000
33/33 [==============================] - 0s 1ms/step - loss: 570481.8295 - val_loss: 530448.6875
Epoch 143/1000
33/33 [==============================] - 0s 1ms/step - loss: 568384.7822 - val_loss: 528382.5000
Epoch 144/1000
33/33 [==============================] - 0s 1ms/step - loss: 566269.2633 - val_loss: 526305.1250
Epoch 145/1000
33/33 [==============================] - 0s 1ms/step - loss: 564144.9337 - val_loss: 524227.7812
Epoch 146/1000
33/33 [==============================] - 0s 1ms/step - loss: 562016.4233 - val_loss: 522154.1250
Epoch 147/1000
33/33 [==============================] - 0s 1ms/step - loss: 559896.7235 - val_loss: 520090.2500
Epoch 148/1000
33/33 [==============================] - 0s 1ms/step - loss: 557767.6629 - val_loss: 517998.0938
Epoch 1

33/33 [==============================] - 0s 1ms/step - loss: 236732.8845 - val_loss: 206307.7031
Epoch 287/1000
33/33 [==============================] - 0s 1ms/step - loss: 234665.8731 - val_loss: 204313.9844
Epoch 288/1000
33/33 [==============================] - 0s 1ms/step - loss: 232592.2902 - val_loss: 202372.1719
Epoch 289/1000
33/33 [==============================] - 0s 1ms/step - loss: 230558.8116 - val_loss: 200415.0625
Epoch 290/1000
33/33 [==============================] - 0s 1ms/step - loss: 228538.7292 - val_loss: 198442.6250
Epoch 291/1000
33/33 [==============================] - 0s 1ms/step - loss: 226474.8191 - val_loss: 196457.9844
Epoch 292/1000
33/33 [==============================] - 0s 1ms/step - loss: 224434.6626 - val_loss: 194491.8906
Epoch 293/1000
33/33 [==============================] - 0s 1ms/step - loss: 222371.6733 - val_loss: 192548.2031
Epoch 294/1000
33/33 [==============================] - 0s 1ms/step - loss: 220346.7027 - val_loss: 190561.7969
Epoch 2

33/33 [==============================] - 0s 1ms/step - loss: 48024.3762 - val_loss: 29079.5117
Epoch 434/1000
33/33 [==============================] - 0s 1ms/step - loss: 47478.4746 - val_loss: 28593.6816
Epoch 435/1000
33/33 [==============================] - 0s 2ms/step - loss: 46940.7633 - val_loss: 28118.5488
Epoch 436/1000
33/33 [==============================] - 0s 2ms/step - loss: 46417.5768 - val_loss: 27656.1582
Epoch 437/1000
33/33 [==============================] - 0s 2ms/step - loss: 45894.9593 - val_loss: 27178.1523
Epoch 438/1000
33/33 [==============================] - 0s 2ms/step - loss: 45379.8811 - val_loss: 26700.3379
Epoch 439/1000
33/33 [==============================] - 0s 1ms/step - loss: 44846.8087 - val_loss: 26254.1445
Epoch 440/1000
33/33 [==============================] - 0s 1ms/step - loss: 44349.5172 - val_loss: 25817.8242
Epoch 441/1000
33/33 [==============================] - 0s 2ms/step - loss: 43869.1863 - val_loss: 25396.0273
Epoch 442/1000
33/33 [===

Epoch 508/1000
33/33 [==============================] - 0s 2ms/step - loss: 20839.5207 - val_loss: 5744.1997
Epoch 509/1000
33/33 [==============================] - 0s 1ms/step - loss: 20679.0239 - val_loss: 5600.6113
Epoch 510/1000
33/33 [==============================] - 0s 1ms/step - loss: 20505.9096 - val_loss: 5457.2627
Epoch 511/1000
33/33 [==============================] - 0s 1ms/step - loss: 20317.7749 - val_loss: 5315.3989
Epoch 512/1000
33/33 [==============================] - 0s 2ms/step - loss: 20145.6097 - val_loss: 5176.6812
Epoch 513/1000
33/33 [==============================] - 0s 1ms/step - loss: 19967.3365 - val_loss: 5051.4653
Epoch 514/1000
33/33 [==============================] - 0s 2ms/step - loss: 19808.0369 - val_loss: 4934.3579
Epoch 515/1000
33/33 [==============================] - 0s 2ms/step - loss: 19661.0938 - val_loss: 4830.8857
Epoch 516/1000
33/33 [==============================] - 0s 1ms/step - loss: 19526.6674 - val_loss: 4740.9185
Epoch 517/1000
33/3

In [36]:
# Predict and measure RMSE
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Score (RMSE): {}".format(score))

Score (RMSE): 33.28727571276187


In [37]:
pred

array([[ 833.08392334],
       [ 833.0838623 ],
       [ 833.08392334],
       [ 833.08392334],
       [ 833.0838623 ],
       [ 833.0838623 ],
       [ 833.0838623 ],
       [ 833.08392334],
       [ 833.08392334],
       [ 833.0838623 ],
       [ 833.0838623 ],
       [ 833.08392334],
       [ 833.08392334],
       [ 833.08392334],
       [ 833.08392334],
       [ 833.08392334],
       [ 833.0838623 ],
       [ 833.0838623 ],
       [ 833.08392334],
       [ 833.0838623 ],
       [ 833.0838623 ],
       [ 833.0838623 ],
       [ 833.0838623 ]], dtype=float32)

In [39]:
df.head(5)

,rank,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_avg_weighted,three_year_ago_avg,three_year_ago_avg_weighted,country_full_class,country_abrv_class,confederation_class,rank_date_class
0,28,791.44,777,2.0,335.41,335.41,430.01,215.01,597.61,179.28,308.68,61.74,0,0,0,0
1,31,741.45,791,-3.0,290.69,290.69,380.25,190.13,604.26,181.28,396.74,79.35,0,0,0,1
2,34,719.61,741,-3.0,298.14,298.14,308.72,154.36,634.66,190.40,383.53,76.71,0,0,0,2
3,34,720.85,720,0.0,285.53,285.53,358.73,179.37,583.78,175.13,404.11,80.82,0,0,0,3
4,34,720.85,721,0.0,285.53,285.53,358.73,179.37,583.78,175.13,404.11,80.82,0,0,0,4
